## Projeto Scraping BeautifulSoup Site CoinMarketCap

In [1]:
from bs4 import BeautifulSoup
import json
import requests
import re

# efetuando conexao com o site
link = "https://coinmarketcap.com/"
req = requests.get(link)

site = BeautifulSoup(req.text, "html.parser")

In [2]:
# encontrando a tabela de dados das moedas
tbody = site.find('tbody')
if not tbody:
    raise Exception("Tabela de moedas não encontrada!")

tlines = tbody.find_all('tr')
if not tlines:
    raise Exception("Nenhuma linha encontrada na tabela!")

coins = {}

for line in tlines:
    try:
        # verificação necessaria para evitar espaços e quebras de linhas desnecessários
        name_coin = line.find(class_="coin-item-name").get_text(strip=True)
        code_coin = line.find(class_="coin-item-symbol").get_text(strip=True)

        # uso de expressões regulares para conseguir os valores que contenham tanto '$ e '%'
        values_coins = line.find_all(string=re.compile(r"\$"))
        percents_variation_coins = line.find_all(string=re.compile("%"))

        '''
            Garantindo que os valores como Preço atual, Market Cap, Oferta circulante, Volume em 24 horas, Variação em 1h,
            Variação em 24 horas e Variação em 7 dias existam na coleta de dados
        '''
        if len(values_coins) < 4 or len(percents_variation_coins) < 3:
            continue # pula para a proxima moeda, caso não exista dados suficientes

        price_coin = values_coins[0]

        # tratamento dos dados com variações percentuais
        for i, percent in enumerate(percents_variation_coins):
            try:
                # print(percent.parent.contents[0]['class'])
                # realizando tratativa dos percentuais conforme site
                if 'icon-Caret-down' in percent.parent.contents[0]['class']:
                    percents_variation_coins[i] = "-" + percent
            except (AttributeError, IndexError, TypeError):
                continue

        # variações percentuais
        variation_1h = percents_variation_coins[0]
        variation_24h = percents_variation_coins[1]
        variation_7d = percents_variation_coins[2]

        marketcap = values_coins[2]
        volume_24hrs = values_coins[3]

        dict_coins = {
            "moeda": name_coin,
            "codigo": code_coin,
            "preço": price_coin,
            "variação_1h": variation_1h,
            "variação_24h": variation_24h,
            "variação_7d": variation_7d,
            "marketcap": marketcap,
            "volume_24hrs": volume_24hrs
        }

        coins[name_coin] = dict_coins
    except AttributeError:
        continue

# salvandso os dados em um arquivo em JSON
with open("dados_criptomoedas.json", "w", encoding="utf-8") as arquivo:
    json.dump(coins, arquivo, indent=4, ensure_ascii=False)

print("Dados enctraídos e salvos com sucesso! ✅ 💾")

Dados enctraídos e salvos com sucesso! ✅ 💾
